In [2]:
import argparse
import pandas as pd
import numpy as np

from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt

#LocalAlignment Implemented Using Smith-Waterman
def LocalAlignment(match_reward: int, mismatch_penalty: int, indel_penalty: int,
                    s: str, t: str):
    rows, cols = len(s) + 1, len(t) + 1
    score_matrix = [[0] * cols for _ in range(rows)]

    maxScore = 0
    maxI, maxJ = 0, 0

    # Fill in the matrix based on match, mismatch, and gap penalties
    for i in range(1, rows):
        for j in range(1, cols):
            match_mismatch_score = match_reward if s[i - 1] == t[j - 1] else (-mismatch_penalty)

            diagonal = score_matrix[i - 1][j - 1] + match_mismatch_score
            left = score_matrix[i][j - 1] - indel_penalty
            up = score_matrix[i - 1][j] - indel_penalty

            score_matrix[i][j] = max(diagonal, left, up, 0)
            if score_matrix[i][j] > maxScore:
                maxScore = score_matrix[i][j]
                maxI, maxJ = i, j
        
    
    alignment1, alignment2 = '', ''
    i, j = maxI, maxJ

    while i > 0 and j > 0 and score_matrix[i][j] > 0:
        if score_matrix[i][j] == score_matrix[i - 1][j - 1] + (match_reward if s[i - 1] == t[j - 1] else (-mismatch_penalty)):
            alignment1 = s[i - 1] + alignment1
            alignment2 = t[j - 1] + alignment2
            i -= 1
            j -= 1
        elif score_matrix[i][j] == score_matrix[i - 1][j] - indel_penalty:
            alignment1 = s[i - 1] + alignment1
            alignment2 = '-' + alignment2
            i -= 1
        else:
            alignment1 = '-' + alignment1
            alignment2 = t[j - 1] + alignment2
            j -= 1
    
    return [maxScore,alignment1,alignment2]

def FastaParse(filename):
    seqDict = {}
    with open(filename, 'r') as file:
        line = file.readline().strip()
        while len(line) != 0:
            if line[0 == '>']:
                seqDict[line] = file.readline().strip()
            line = file.readline().strip()
    file.close()
    return seqDict



In [3]:

filename = 'test_data/pone.0192851.s009.faa'
seqs = FastaParse(filename)

m,s,d = 1, -10, -2

alnScores = {}
for seq in seqs:
    alnScores[seq] = {}
    toPopulate = {other for other in seqs if other != seq}
    for i in toPopulate:
        alnScores[seq][i] = {'Score' : None, 'Alignment' : None}

for seq in alnScores:
    for i in alnScores[seq]:
        if alnScores[seq][i]['Score'] == None and alnScores[seq][i]['Alignment'] == None:
            print('aligning ' +seq +' with ' + i)
            
            score, alignment1, alignment2 = LocalAlignment(m,s,d,seqs[seq],seqs[i])
            alnScores[seq][i] = {'Score' : score, 'Alignment' : alignment1}
            alnScores[i][seq] = {'Score' : score, 'Alignment' : alignment2}

#print(seqs)
#print(alnScores)
print('DONE')
scores = {}
for outer_key, inner_dict in alnScores.items():
    scores[outer_key] = {inner_key: values['Score'] for inner_key, values in inner_dict.items()}
df = pd.DataFrame(scores)


aligning >1.A.17.1.1 with >1.A.17.2.5
aligning >1.A.17.1.1 with >1.A.17.4.5
aligning >1.A.17.1.1 with >1.A.17.4.7
aligning >1.A.17.1.1 with >1.A.17.4.6
aligning >1.A.17.1.1 with >1.A.17.5.12
aligning >1.A.17.1.1 with >1.A.17.1.16
aligning >1.A.17.1.1 with >1.A.17.6.2
aligning >1.A.17.1.1 with >1.A.17.5.7
aligning >1.A.17.1.1 with >1.A.17.1.13
aligning >1.A.17.1.1 with >1.A.17.5.2
aligning >1.A.17.1.1 with >1.A.17.2.1
aligning >1.A.17.1.1 with >1.A.17.2.4
aligning >1.A.17.1.1 with >1.A.17.4.8
aligning >1.A.17.1.1 with >1.A.17.6.4
aligning >1.A.17.1.1 with >1.A.17.4.4
aligning >1.A.17.1.1 with >1.A.17.6.3
aligning >1.A.17.1.1 with >1.A.17.1.18
aligning >1.A.17.1.1 with >1.A.17.1.17
aligning >1.A.17.1.1 with >1.A.17.3.2
aligning >1.A.17.1.1 with >1.A.17.7.2
aligning >1.A.17.1.1 with >1.A.17.5.11
aligning >1.A.17.1.1 with >1.A.17.3.10
aligning >1.A.17.1.1 with >1.A.17.2.3
aligning >1.A.17.1.1 with >1.A.17.7.1
aligning >1.A.17.1.1 with >1.A.17.5.1
aligning >1.A.17.1.1 with >1.A.17.1.12
alig

In [9]:
'''
# Step 1: Force Symmetry by averaging the upper and lower triangles
def make_symmetric(df):
    return (df + df.T) / 2

df_symmetric = make_symmetric(df)

# Verify symmetry (just for confirmation)
assert np.allclose(df_symmetric, df_symmetric.T), "DataFrame is not symmetric"
'''

df = pd.read_csv('alnScoreMatrixSorted.csv')
print(df)

assert np.allclose(df, df.T), "DataFrame is not symmetric"

      Unnamed: 0  >1.A.17.1.1  >1.A.17.1.10  >1.A.17.1.11  >1.A.17.1.12  \
0    >1.A.17.1.1          0.0           6.0           5.0           6.0   
1   >1.A.17.1.10          6.0           0.0           5.0           5.0   
2   >1.A.17.1.11          5.0           5.0           0.0           5.0   
3   >1.A.17.1.12          6.0           5.0           5.0           0.0   
4   >1.A.17.1.13          5.0           6.0           5.0           5.0   
..           ...          ...           ...           ...           ...   
64   >1.A.17.7.1          4.0           4.0           4.0           5.0   
65   >1.A.17.7.2          4.0           4.0           5.0           5.0   
66   >1.A.17.7.3          5.0           4.0           4.0           4.0   
67   >1.A.17.7.4          4.0           5.0           4.0           5.0   
68   >1.A.17.7.5          5.0           4.0           4.0           6.0   

    >1.A.17.1.13  >1.A.17.1.14  >1.A.17.1.15  >1.A.17.1.16  >1.A.17.1.17  ...  \
0            5.0  

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [8]:
max_score = np.max(df.values)
distance_matrix = max_score - df.values

TypeError: '>=' not supported between instances of 'str' and 'float'

In [6]:
# Step 2: Perform Hierarchical Clustering
# Convert the DataFrame to a condensed distance matrix, required by linkage
condensed_distance_matrix = squareform(df.values)
Z = linkage(condensed_distance_matrix, method='average')  # You can change the method as needed

# Step 3: Generate and Save the Tree
plt.figure(figsize=(10, 7))
dendrogram(Z, labels=df.index.tolist())
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sequences')
plt.ylabel('Distance')
plt.show()

# Save the plot as an image
plt.savefig('hierarchical_clustering_tree.png')

ValueError: Distance matrix 'X' must be symmetric.